In [1]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from geopy.distance import geodesic


In [2]:
df = pd.read_csv('cs_data_2.csv')

In [3]:
df=df[df['city_name']=='Bengaluru']
df=df[df['order_status']==6]
df=df[df['driver_type']==15]
df=df[df['predicted_food_prep_time'].notnull()]
df=df[df['pickup_to_drop_location_km'].notnull()]
df=df[df['expected_delivery_time'].notnull()]
df=df[df['pickup_latitude'].notnull()]
df=df[df['pickup_longitude'].notnull()]
df=df[df['drop_latitude'].notnull()]
df=df[df['drop_longitude'].notnull()]

In [4]:
df.head(5)

,food_order_id,food_restaurant_id,pickup_latitude,pickup_longitude,city_name,drop_latitude,drop_longitude,created_at,pickup_to_drop_location_km,expected_delivery_time,predicted_food_prep_time,driver_type,order_status,food_item_name,food_item_quantity,food_item_unit_cost
17,53849642430,1766072690,3.042535,92.623123,Bengaluru,3.057000,92.627000,2020-02-01 20:27:25.000,3.951,36.0,19.48,15,6,Guntur Chicken Fry,1,449.0
18,53849642430,1766072690,3.042535,92.623123,Bengaluru,3.057000,92.627000,2020-02-01 20:27:25.000,3.951,36.0,19.48,15,6,Boneless Chicken Biryani,1,379.0
19,53849642430,1766072690,3.042535,92.623123,Bengaluru,3.057000,92.627000,2020-02-01 20:27:25.000,3.951,36.0,19.48,15,6,Chicken 65,1,389.0
20,55588091162,1766072690,3.042535,92.623123,Bengaluru,3.055996,92.635498,2020-02-01 21:12:50.000,2.997,32.0,13.28,15,6,Mutton Dum Biryani,1,259.0
21,55839822095,1766072690,3.042535,92.623123,Bengaluru,3.038721,92.620271,2020-02-01 13:40:14.000,0.409,22.0,11.25,15,6,Ancient Hyderabad Special Paneer Biryani,1,209.0


In [5]:
df['created_at'] = df['created_at'].astype(str).str.split(".", n = 1, expand = True)[0]
new = df["predicted_food_prep_time"].astype(str).str.split(".", n = 1, expand = True)
df["Minutes"] = new[0]
df["Seconds"] = new[1]
df["created_time"] = df["created_at"].astype(str).str.split(" ", n = 1, expand = True)[1]
df['delivery_time'] = (df['pickup_to_drop_location_km']/15)*60 #in minutes
df['predicted_food_prep_timestamp'] = pd.to_datetime(df.created_at) + pd.to_timedelta(df.Minutes.astype(int),unit='m') + pd.to_timedelta(df.Seconds.astype(int),unit='s')# + pd.to_timedelta(df.Seconds, unit='s')
df['buffer_time'] = (df['expected_delivery_time'] - df['predicted_food_prep_time'] - df['delivery_time']).apply(np.floor)


In [6]:
df.head(5)

,food_order_id,food_restaurant_id,pickup_latitude,pickup_longitude,city_name,drop_latitude,drop_longitude,created_at,pickup_to_drop_location_km,expected_delivery_time,...,order_status,food_item_name,food_item_quantity,food_item_unit_cost,Minutes,Seconds,created_time,delivery_time,predicted_food_prep_timestamp,buffer_time
17,53849642430,1766072690,3.042535,92.623123,Bengaluru,3.057000,92.627000,2020-02-01 20:27:25,3.951,36.0,...,6,Guntur Chicken Fry,1,449.0,19,48,20:27:25,15.804,2020-02-01 20:47:13,0.0
18,53849642430,1766072690,3.042535,92.623123,Bengaluru,3.057000,92.627000,2020-02-01 20:27:25,3.951,36.0,...,6,Boneless Chicken Biryani,1,379.0,19,48,20:27:25,15.804,2020-02-01 20:47:13,0.0
19,53849642430,1766072690,3.042535,92.623123,Bengaluru,3.057000,92.627000,2020-02-01 20:27:25,3.951,36.0,...,6,Chicken 65,1,389.0,19,48,20:27:25,15.804,2020-02-01 20:47:13,0.0
20,55588091162,1766072690,3.042535,92.623123,Bengaluru,3.055996,92.635498,2020-02-01 21:12:50,2.997,32.0,...,6,Mutton Dum Biryani,1,259.0,13,28,21:12:50,11.988,2020-02-01 21:26:18,6.0
21,55839822095,1766072690,3.042535,92.623123,Bengaluru,3.038721,92.620271,2020-02-01 13:40:14,0.409,22.0,...,6,Ancient Hyderabad Special Paneer Biryani,1,209.0,11,25,13:40:14,1.636,2020-02-01 13:51:39,9.0


In [7]:
total_orders = len(df.groupby('food_order_id'))
print("Total orders: ", end = " ")
print(total_orders)
#print(len(df['food_order_id'].unique()))
df_orders = df.groupby('food_restaurant_id')
#print(type(df_orders))
#print(len(df_orders.groups))
batched_orders = 0



Total orders:  131271


In [8]:
print(len(df_orders.groups.keys()))

6991


In [9]:
batch_possibility = {}

In [10]:
temp =0  #comment this line to run for entire dataset (all restaurant ids)
for name,group in df_orders:
    restaurant_id = name #comment this line to run for entire dataset (all restaurant ids)
    temp = 1 #comment this line to run for entire dataset (all restaurant ids)
    df_orders_restaurant = df_orders.get_group(name).reset_index()
    df_orders_group = df_orders_restaurant.groupby('food_order_id')
    total_orders = len(df_orders_group) #comment this line to run for entire dataset (all restaurant ids)
    already_batched = []
    for order1,data1 in df_orders_group:
        possible_order2 = []
        if order1 in already_batched:
            continue
        dist_RA = geodesic((data1.pickup_latitude.get(data1.pickup_latitude.index[0]),data1.pickup_longitude.get(data1.pickup_longitude.index[0])), (data1.drop_latitude.get(data1.drop_latitude.index[0]),data1.drop_longitude.get(data1.drop_longitude.index[0]))).km
        time_RA = dist_RA/15*60 #in minutes
        for order2,data2 in df_orders_group:
            dist_AB = geodesic((data1.drop_latitude.get(data1.drop_latitude.index[0]),data1.drop_longitude.get(data1.drop_longitude.index[0])), (data2.drop_latitude.get(data2.drop_latitude.index[0]),data2.drop_longitude.get(data2.drop_longitude.index[0]))).km
            time_AB = dist_AB/15*60
            dist_RB = geodesic((data1.pickup_latitude.get(data1.pickup_latitude.index[0]),data1.pickup_longitude.get(data1.pickup_longitude.index[0])), (data2.drop_latitude.get(data2.drop_latitude.index[0]),data2.drop_longitude.get(data2.drop_longitude.index[0]))).km
            time_RB = dist_RB/15*60
            order_checkout_time = max(data1.predicted_food_prep_timestamp.get(data1.predicted_food_prep_timestamp.index[0]),data2.predicted_food_prep_timestamp.get(data2.predicted_food_prep_timestamp.index[0]))
            if order1==order2:
                continue
            if (abs((data2.predicted_food_prep_timestamp.get(data2.predicted_food_prep_timestamp.index[0]) - data1.predicted_food_prep_timestamp.get(data1.predicted_food_prep_timestamp.index[0])).total_seconds()/60.0) <= data1.buffer_time.get(data1.buffer_time.index[0])):
                possible_order2.append(order2)
                #if the below condition is true that means it doesn't satisfy RA+AB<AR+RB, so we remove the order2 from batching possibility
                if  dist_RA + dist_AB > dist_RA + dist_RB: #locations in opposite direction and hence impossible to batch
                    possible_order2.remove(order2)
                    continue
                    #break
                #if the below condition is true that means at any one order will have a breach, so we remove the order2 from batching possibility
                if (dt.timedelta(data1.expected_delivery_time.get(data1.expected_delivery_time.index[0])) < pd.to_timedelta(str(order_checkout_time).split()[1]) + pd.to_timedelta(time_RA,unit='m')):
                    possible_order2.remove(order2)
                    continue
                if (dt.timedelta(data2.expected_delivery_time.get(data2.expected_delivery_time.index[0])) < pd.to_timedelta(str(order_checkout_time).split()[1]) + pd.to_timedelta(time_RA,unit='m') + pd.to_timedelta(time_AB,unit='m')):
                    possible_order2.remove(order2)
                if len(possible_order2) == 1:
                    batch_possibility[order1] = order2
                    already_batched.append(order2)
                    break
    if temp == 1: #comment this line to run for entire dataset (all restaurant ids)
        break #comment this line to run for entire dataset (all restaurant ids)

In [11]:
print("For restaurant id", end = " ") #comment this line to run for entire dataset (all restaurant ids)
print(restaurant_id)                  #comment this line to run for entire dataset (all restaurant ids)
print("Total orders: ", end=" ")
print(total_orders)
print("Number of batched orders: ", end=" ")
print(len(batch_possibility)*2)
print("Share of successfully delivered orders that can be batched: ", end=" ")
print((len(batch_possibility)*2)/total_orders*100)


For restaurant id 4653062
Total orders:  94
Number of batched orders:  78
Share of successfully delivered orders that can be batched:  82.97872340425532
